In [ ]:
# cleaning...%load_ext autoreload
# cleaning...%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
signal = pd.read_csv(PATH["feature_x_sample.signal.tsv"], sep="\t", index_col=0)

signal.columns.name = SETTING["sample_alias"]

signal.fillna(value=0, inplace=True)

In [ ]:
k_mfcc = kraft.mf_consensus_cluster_dataframe_with_ks(
    signal,
    SETTING["mf_ks"],
    n_job=SETTING["n_job"],
    n_clustering=SETTING["mf_n_clustering"],
    plot_dataframe=False,
    plot_w=True,
    plot_h=True,
    directory_path=PATH["mf/"],
)

In [ ]:
for k in SETTING["mf_ks"]:

    for w_or_h, axis in (("w", 0), ("h", 1)):

        dataframe = pd.read_csv(
            os.path.join(PATH["mf/"], str(k), "{}.tsv".format(w_or_h)),
            sep="\t",
            index_col=0,
        )

        if w_or_h == "w":

            dataframe.columns.name = "Factor"

        kraft.apply_function_on_slices_from_2_dataframes(
            dataframe,
            dataframe,
            axis,
            kraft.compute_information_coefficient_between_2_vectors,
            layout={"title": {"text": "MF K={} {}".format(k, w_or_h.title())}},
        )